In [1]:
import pandas as pd
import re
import nltk
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from textblob import TextBlob
import wordsegment as ws 
ws.load()
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib

In [2]:
data_politics = pd.read_json('tweets_politics.json', lines = True)
data_politics.sort_values('text',inplace=True)
data_politics.drop_duplicates(subset='text',keep = False, inplace=True)
data_politics.shape

(4469, 37)

In [3]:
def clean_tweets(df):
  i = 1  
  all_tweets = []
  tweets = df['text'].values.tolist();
  for text in tweets:
    print()
    print(i)
    i +=1
    print(text)
    text = text.lower()
    text_pattern = re.compile('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]|))+')
    text = text_pattern.sub('', text)
    emoji = re.compile("["
                        u"\U0001F600-\U0001FFFF"   #EMOJIS
                        u"\U0001F300-\U0001F5FF"   #Symbols and Pictographs
                        u"\U0001F680-\U0001F6FF"   #map symbols
                        u"\U0001F1E0-\U0001F1FF"   #flags
                        u"\U00002702-\U000027B0"
                        u"\U000024C2-\U0001F251"
                        "]+", flags=re.UNICODE)
    text = emoji.sub(r'', text)
    text = re.sub(r'@[a-z0-9]+','',text)
   # text = re.sub(r'#', '', text)
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"this's", "this is", text)
    text = re.sub(r"what's", "what is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"\'ll", "will", text)
    text = re.sub(r"\'ve", "have", text)
    text = re.sub(r"\'m", "am", text)
    text = re.sub(r"\'d", "would", text)
    text = re.sub(r"\'re", "are", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"don't", "do not", text)
    text = re.sub(r"didn't", "did not", text)
    text = re.sub(r"can't", "can not", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"couldn't", "could not", text)
    text = re.sub(r"haven't", "have not", text)
    text = re.sub(r'rt[\s]+', '', text)

    text = re.sub(r"[,.\"!#$%^&*(){}?/;'~:<>+=-]", "", text)
    
    tokens = word_tokenize(text)
    
    text = [ws.clean(w) for w in text if len(ws.clean(w)) > 0]
    text = [ws.segment(w)[0] for w in text]   #segmentation
    
    stop_words = set(stopwords.words("english"))
    stop_words.discard("not")
    table = str.maketrans('','',string.punctuation)
    stripped = [w.translate(table) for w in tokens]
    words = [word for word in stripped if word.isalpha()]

    words = [w for w in words if not w in stop_words]
    words = ' '.join(words)
        
    print(words)
    
    all_tweets.append(words)
  return all_tweets

In [4]:
nan_value = float("NaN")

Cleaned_tweets = clean_tweets(data_politics)

data_politics['text_cleaned'] = Cleaned_tweets

data_politics.replace("", nan_value, inplace=True)
data_politics.dropna(subset=['text_cleaned'], inplace=True)

data_politics = data_politics.sample(frac = 1)

data_politics = data_politics[data_politics['text_cleaned'].str.split().str.len().gt(4)]

data_politics = data_politics[['id', 'text_cleaned','text']]

data_politics.to_csv('tweets_cleaned_politics.csv',index=False)


1
!!! get your political news &amp; discussion here! new episode out now!! #HighThoughts #politics #HalleBerry  https://t.co/hg0peLGZv1
get political news amp discussion new episode highthoughts politics halleberry

2
"Biden administration has rolled back a Trump-era regulation meant to protect politically disfavored businesses, li… https://t.co/62bANfNuyR
biden administration rolled back trumpera regulation meant protect politically disfavored businesses

3
"Burnham's Machievellians taught him that politics could always be reduced to the pursuit of power; behind every ap… https://t.co/tXwQW73wV1
burnhams machievellians taught politics could always reduced pursuit power behind every

4
"Democrats and Republicans live apart from each other, down to the neighborhood, to a degree that raises provocativ… https://t.co/CNyDvfSmBu
democrats republicans live apafrom neighborhood degree raises

5
"Happiest people don't have the best of everything...
They just make the best of everything"
:)  :

anildeshmukh seems truth come forward many wickets go defeat

64
#AnilDeshmukh #100cr #100CroreKiVasuli #ParamBirsinghLetter

After #ParambirSingh's Letter, BJP : https://t.co/LXhyuTuK6r
anildeshmukh parambirsinghletter parambirsinghs letter bjp

65
#AnilDeshmukh #MaharashtraGovernment 
Maharastra Government  to M. public https://t.co/AYvAvgqixS
anildeshmukh maharashtragovernment maharastra government public

66
#AnilDeshmukh #ParambirSingh playing dirty games , Ganda hai Par Dhandha hai yeh #100Crore sales target , mock up r… https://t.co/6kHPGgbqXb
anildeshmukh parambirsingh playing dirty games ganda hai par dhandha hai yeh sales target mock

67
#AnilDeshmukh @CMOMaharashtra  be like this https://t.co/mxXz24Shcx
anildeshmukh like

68
#AnilDeshmukh A government busy vadooling Haftas involving police force who has to be in service. They would have v… https://t.co/NRwiPacW1n
anildeshmukh government busy vadooling haftas involving police force service would

69
#AnilDeshmukh This video h

modihaitomehangaihai firstonkoo raga prithviraj many would surprised cabal

158
#ModiHaiToMehangaiHai #fakefakir expected from PM (1) electoral reform (2) special court for tainted MP-MLA (3) Jud… https://t.co/kgaePkqG8L
modihaitomehangaihai fakefakir expected pm electoral reform special coufor tainted mpmla

159
#ModiHaiToMehangaiHai . So why u lied before . That's why the farmers are not trusting on modi. Earlier in 2013 thi… https://t.co/1zw5JBRV15
modihaitomehangaihai u lied farmers not trusting modi earlier

160
#ModiHaiToMehangaiHai Cooking Gas price is at all time high, Rs 50 increased this week itself!
What this govt wants… https://t.co/e5ZaKooo3y
modihaitomehangaihai cooking gas price time high rs increased week govt

161
#ModiHaiToMehangaiHai Corona warrior death 14.06.20 covid hospital safdarjung Delhi, private employee jo general op… https://t.co/snwUfsQepL
modihaitomehangaihai corona warrior death covid hospital safdarjung delhi private employee jo general

162
#ModiHaiToM

sharing without comments petroldieselpricehike

275
#petrol100
This is one issue mentally affecting ones facilities. Though it is alarming it's risky though. I am real… https://t.co/2FWIr2TzYq
one issue mentally affecting ones facilities though alarming risky though

276
#petrol100
Welcome new india https://t.co/TLn4ji6zyk
welcome new india

277
#petrol100
What will u call a citizen who protests against #FuelLoot &amp; questions his Govt over #petrol100?
1… https://t.co/O5KVdDPtJD
u call citizen protests fuelloot amp questions govt

278
#petrol100 

Bargain between Petrol Pump Employee and Customer after Prices touch 200 in future : https://t.co/xWUKdwe30K
bargain petrol pump employee customer prices touch future

279
#petrol100 

Now we need to wait for #2024.

🐨
need wait

280
#petrol100 

This is what a 100 Rupee note can get you in an Indian Village :)

Read this beautiful incident in Hin… https://t.co/3mPGqutVNc
rupee note get indian village read beautiful incident

281
#petrol100

ruppes andhbhakt trending le youth trend meanwhile modi ji amp ministers

399
2. In Nawaz’s third tenure, significant efforts were made to use diplomacy to reengage India on Kashmir and broader… https://t.co/ydPSbhPfxb
nawaz third tenure significant efforts made use diplomacy reengage india kashmir

400
2020 was a tough year for Russia w/ its neighbors. Carol Saivetz, @MIT_SSP Russia expert, examines Russia’s issues… https://t.co/CAGH9SwaR9
tough year russia w neighbors carol saivetz ssp russia expert examines russia

401
215 K tweets done .. bravo guys .   Keep going until we get Another Jumla from CM 

#4_साल_यूपी_बेहाल
k tweets done bravo guys keep going get another jumla cm

402
28 games to choose from! Get in the BundleFest spirit https://t.co/F8EVif5W1l #get #bundlefest #steam #sales… https://t.co/9dwl0mRX5m
games choose get bundlefest spirit get bundlefest steam

403
50rs katto over acting ka 🤣🤣🤣 #IndiaWithModi #petrol100
katto acting ka indiawithmodi

404
56" Chest can observe 

...

supreme leader skilled aof fooling indians modihaitomehngaihai

3962
Sure.
sure

3963
Swiggy delivery executives switch to bicycles as fuel rates ⛽ surge 🤣😜

#petrol100 
 https://t.co/LOYaxojkeF
swiggy delivery executives switch bicycles fuel rates surge

3964
Synonym words -- BJP and Hate !!

#FreeDishaRavi 
#ModiHaiToMehngaiHai 
#BJP 
#petrol100 
#PetrolPriceHike… https://t.co/KClVkApwFW
synonym words bjp hate freedisharavi modihaitomehngaihai bjp

3965
T890
Appreciable
But will it reduces our burden
appreciable reduces burden

3966
THE GREAT INDIAN GURU#petrol100 https://t.co/z3hOiZO0La
great indian

3967
TMC knows its fate.@narendramodi @BJP4India @JPNadda
tmc knows fate

3968
TODAY Drew Allen joins @rushtoreason to discuss the new Republican Party #Republicans #RepublicanParty #politics… https://t.co/G1AAKd1EJt
today drew allen joins discuss new republican party republicans republicanparty

3969
Taiwan says BioNTech vaccine deal on hold https://t.co/Bg2enGET4H #Politics https://t.

farm bills result pricerise essential commodities not fight farmers

4066
These are exactly the words that should be used to describe #NordStream2. And exactly the course of action that wil… https://t.co/ZnZKeMZt8t
exactly words used describe exactly course action

4067
These are not politicians but they uses family n person name. Their fathers made money by that they can abuse anyon… https://t.co/rWYfQlEa6l
not politicians uses family n person name fathers made money abuse

4068
These guys were caught in lies.
#pinocchioQOPS

https://t.co/DxrdlkeBI8
guys caught lies pinocchioqops

4069
These other companies’ CBD products were said to be available at health and wellness centers, nationally. #Putin
companies cbd products said available health wellness centers nationally putin

4070
These shows how #politics changed in #Telangana #India . The convict is of trspartyonline and a close relationship… https://t.co/spi59Jjw8d
shows politics changed telangana india convict trspartyonline close


choose answer poll comments modihaitomehngaihai farmersprotest

4182
Wah, Email address hasn't verify and home minister #AnilDeshmukh said he will be filing Defamation suit against… https://t.co/6F8tHXguNr
wah email address hasnt verify home minister anildeshmukh said filing defamation suit

4183
Wait for some time before it becomes "Adani Oil &amp; Gas"
#ModiHaiToMehngaiHai
wait time becomes adani oil amp gas modihaitomehngaihai

4184
Waiting for #sanjayRaut opinion who always has supported Mumbai Police &amp; has some opinion everytime, Whom will he s… https://t.co/FqBa3EiHuF
waiting sanjayraut opinion always supported mumbai police amp opinion everytime

4185
Waiting to celebrate the century .
#petrol100
waiting celebrate century

4186
Want to know the Winners in Modern Elections? https://t.co/xUr2vh9uNX #election #Election2020results #Politics… https://t.co/9TSjDm4LK2
want know winners modern elections election

4187
Want to know what #CargoCult is? Join @ConCaracal #live to see ho

wherein putin threatens president united states

4298
Which news channel had dared to debate when 'IAS' 'Bk Bansal' had named Amit shah's in his suicide letter. #AnilDeshmukh #ParambirSingh
news channel dared debate ias bk bansal named amit shahs suicide letter anildeshmukh parambirsingh

4299
White House refuses to call Saudi leader MBS a 'killer' after Biden called Putin one https://t.co/Hym9Kl9zeG https://t.co/WK01TAxLSF
white house refuses call saudi leader mbs killer biden called putin one

4300
White House says Biden has no regrets in calling Putin a 'killer'
Well Son Of Bitch Joe Biden is going to get USA n… https://t.co/ehIT99SI6F
white house says biden regrets calling putin killer well son bitch joe biden going get usa

4301
Who bothers? 
#petrol100 #IPLAuction2021 #FarmersProtest #COVID19 https://t.co/RTVYshCYzG
bothers farmersprotest

4302
Who did you vote for ? 🇺🇸
https://t.co/ySvtfJLw7j

#politics #biden #trump #fakeart #america https://t.co/FaCLaigJ5I
vote politics biden 

via joebiden politics business usnews aptopnews biden wage proposal job killer

4424
via @PerilOfAfrica #JoeBiden #Politics #USNews #Pandemics Biden reframes his goal on reopening of elementary schools https://t.co/O4k27wXOA9
via joebiden politics usnews pandemics biden reframes goal reopening elementary schools

4425
wait what does that mean i'm stupid :/
wait mean stupid

4426
well done it a century guys
#petrol100
well done century guys

4427
what is the punishment for this gangs of Wazepur in:-
1. india
2. russia
3. China
punishment gangs wazepur india russia china

4428
yall from the US be easy🙏
yall us easy

4429
yeah putin's a killer, killin these bitches
yeah putins killer killin bitches

4430
अब की बार Middle class पर वार !
-No Income Tax relief
-No Health care relief
-No Education fee relief
-No EMI relie… https://t.co/91AVkzOJV4
अब middle class पर income tax relief health care relief education fee relief emi

4431
अब की बार Middle class पर वार !
-No Income Tax relief
-No Hea